In [2]:
import gensim
from glob import glob
from lxml import html

In [4]:
collection = []
for filename in glob.glob('/Users/tiffanyfrench/reut2-*'):
    with open(filename) as fd:
        try:
            _ = html.fromstring(fd.read())
        except UnicodeDecodeError:
            print(f'skipped "{filename}"')
            continue

        for doc in _.body.iter('text'):
            text = doc.text_content()
            clean = gensim.utils.simple_preprocess(text)
            collection.append(clean)

AttributeError: 'function' object has no attribute 'glob'

In [7]:
id2word = gensim.corpora.Dictionary(collection)

In [8]:
id2word.filter_extremes(no_below=20, no_above=.1)

In [9]:
bow_collection = list(map(id2word.doc2bow, collection))

In [10]:
lda = gensim.models.ldamodel.LdaModel(
    corpus=bow_collection,
    id2word=id2word,
    num_topics=100,
)

ValueError: cannot compute LDA over an empty collection (no terms)

In [113]:
one = '''
Whales are creatures of the open ocean; they feed, mate, give birth, 
suckle and raise their young at sea. So extreme is their adaptation 
to life underwater that they are unable to survive on land. Whales 
range in size from the 2.6 metres (8.5 ft) and 135 kilograms (298 lb) 
dwarf sperm whale to the 29.9 metres (98 ft) and 190 metric tons 
(210 short tons) blue whale, which is the largest creature that has 
ever lived. The sperm whale is the largest toothed predator on earth. 
Several species exhibit sexual dimorphism, in that the females are 
larger than males. Baleen whales have no teeth; instead they have 
plates of baleen, a fringe-like structure used to expel water while 
retaining the krill and plankton which they feed on. They use their 
throat pleats to expand the mouth to take in huge gulps of water. 
Balaenids have heads that can make up 40% of their body mass to take 
in water. Toothed whales, on the other hand, have conical teeth 
adapted to catching fish or squid. Baleen whales have a well 
developed sense of "smell", whereas toothed whales have 
well-developed hearing − their hearing, that is adapted for both air 
and water, is so well developed that some can survive even if they 
are blind. Some species, such as sperm whales, are well adapted for 
diving to great depths to catch squid and other favoured prey.
'''

In [116]:
?lda.get_document_topics

In [119]:
bow = id2word.doc2bow(gensim.utils.simple_preprocess(one))
topics = lda.get_document_topics(bow, minimum_probability=0)

In [117]:
for key, contribution in topics:
    print(f'\n:: {key} ({contribution*100:.2}%)')
    for i, __ in lda.get_topic_terms(key, topn=5):
        print(f' {id2word[i]}', end=',')


:: 0 (0.013%)
 stg, london, plc, england, assistance,
:: 1 (0.013%)
 sales, days, second, car, phelps,
:: 2 (3.6%)
 blah, says, ton, ore, tons,
:: 3 (3.0%)
 sterling, start, link, night, fairchild,
:: 4 (0.013%)
 savings, federal, association, home, loan,
:: 5 (0.013%)
 ltd, bond, lead, london, morgan,
:: 6 (0.013%)
 contract, aluminium, gets, metal, awarded,
:: 7 (0.013%)
 record, dividend, pay, div, prior,
:: 8 (0.013%)
 warrants, issue, each, price, receipts,
:: 9 (0.013%)
 securities, commission, sec, usair, twa,
:: 10 (0.013%)
 court, texas, against, state, district,
:: 11 (2.3%)
 technology, sea, ships, anti, investigation,
:: 12 (0.013%)
 acquisition, group, pacific, sale, american,
:: 13 (0.013%)
 stake, gas, group, investment, natural,
:: 14 (0.013%)
 mexico, lawson, buffer, pact, mexican,
:: 15 (0.013%)
 saudi, palm, shipment, arabia, malaysian,
:: 16 (0.013%)
 west, inflation, growth, german, prices,
:: 17 (0.013%)
 debt, analysts, moody, standard, cash,
:: 18 (0.013%)
 el,

In [120]:
??lda.get_document_topics

In [121]:
??lda.inference

In [122]:
bow = id2word.doc2bow(gensim.utils.simple_preprocess(one))
gamma, phi = lda.inference([bow])
one_topics = gamma[0] / gamma[0].sum()
one_topics

array([1.2829622e-04, 1.2829622e-04, 3.5820153e-02, 3.0075325e-02,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 2.2704571e-02,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       1.7070265e-02, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       4.8054587e-02, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 1.6206250e-02,
       1.2829622e-04, 1.2829622e-04, 7.0734799e-02, 1.2829622e-04,
       1.2829622e-04, 3.0298945e-02, 1.2829622e-04, 1.4112905e-02,
       1.2829622e-04, 1.2829622e-04, 1.2829622e-04, 1.2829622e-04,
       1.2829622e-04, 1.4834500e-02, 1.2829622e-04, 1.2829622e

In [123]:
two = """
Whales have evolved from land-living mammals. As such whales must 
breathe air regularly, although they can remain submerged under 
water for long periods of time. Some species such as the sperm whale 
are able to stay submerged for as much as 90 minutes.[1] They have 
blowholes (modified nostrils) located on top of their heads, through 
which air is taken in and expelled. They are warm-blooded, and have 
a layer of fat, or blubber, under the skin. With streamlined fusiform 
bodies and two limbs that are modified into flippers, whales can 
travel at up to 20 knots, though they are not as flexible or agile as 
seals. Whales produce a great variety of vocalizations, notably the 
extended songs of the humpback whale. Although whales are widespread, 
most species prefer the colder waters of the Northern and Southern 
Hemispheres, and migrate to the equator to give birth. Species such as 
humpbacks and blue whales are capable of travelling thousands of miles 
without feeding. Males typically mate with multiple females every year, 
but females only mate every two to three years. Calves are typically 
born in the spring and summer months and females bear all the 
responsibility for raising them. Mothers of some species fast and nurse 
their young for one to two years.
"""

bow = id2word.doc2bow(gensim.utils.simple_preprocess(two))
gamma, phi = lda.inference([bow])
two_topics = gamma[0] / gamma[0].sum()
two_topics

array([1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 3.07984333e-02,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.43771227e-02,
       1.37087176e-04, 1.37087176e-04, 6.46264702e-02, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
       1.45604368e-02, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.56509921e-01,
       2.00531352e-02, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 3.13944630e-02, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
       1.37087176e-04, 1.37087176e-04, 1.37087176e-04, 1.37087176e-04,
      

In [124]:
three = """
Curiosity is a car-sized rover designed to explore the crater Gale on 
Mars as part of NASA's Mars Science Laboratory mission (MSL).[3] 
Curiosity was launched from Cape Canaveral on November 26, 2011, at 
15:02 UTC and landed on Aeolis Palus inside Gale on Mars on August 6, 
2012, 05:17 UTC.[7][8][13] The Bradbury Landing site was less than 2.4 
km (1.5 mi) from the center of the rover's touchdown target after a 560 
million km (350 million mi) journey.[9][14] The rover's goals include 
an investigation of the Martian climate and geology; assessment of 
whether the selected field site inside Gale has ever offered 
environmental conditions favorable for microbial life, including 
investigation of the role of water; and planetary habitability studies 
in preparation for human exploration.[15][16]

In December 2012, Curiosity's two-year mission was extended indefinitely,
[17] and on August 5, 2017, NASA celebrated the fifth anniversary of the 
Curiosity rover landing.[18][19] The rover is still operational, and as 
of February 15, 2019, Curiosity has been on Mars for 2321 sols (2384 
total days) since landing on August 6, 2012. (See current status.)

Curiosity's design serves as the basis for the planned Mars 2020 rover, 
that will carry different scientific instruments.
"""

bow = id2word.doc2bow(gensim.utils.simple_preprocess(three))
gamma, phi = lda.inference([bow])
three_topics = gamma[0] / gamma[0].sum()
three_topics

array([0.00014504, 0.11121283, 0.01971041, 0.00014504, 0.00014504,
       0.00014504, 0.00014504, 0.00014504, 0.00014504, 0.00014504,
       0.01861148, 0.00014504, 0.00014504, 0.00014504, 0.04049384,
       0.00014504, 0.00014504, 0.00014504, 0.08689582, 0.00014504,
       0.00014504, 0.00014504, 0.00014504, 0.00014504, 0.00014504,
       0.00014504, 0.00014504, 0.08127526, 0.00014504, 0.00014504,
       0.00014504, 0.00014504, 0.00014504, 0.00014504, 0.02468311,
       0.00014504, 0.00014504, 0.00014504, 0.00014504, 0.00014504,
       0.00014504, 0.01928862, 0.00014504, 0.00014504, 0.00014504,
       0.00014504, 0.08799753, 0.00014504, 0.00014504, 0.00014504,
       0.00014504, 0.00014504, 0.00014504, 0.00014504, 0.00014504,
       0.00014504, 0.04051079, 0.02627189, 0.01621567, 0.00014504,
       0.00014504, 0.00014504, 0.00014504, 0.00014504, 0.00014504,
       0.00014504, 0.00014504, 0.04216   , 0.03285354, 0.07170033,
       0.04113631, 0.04738518, 0.01583654, 0.00014504, 0.00014

In [125]:
import scipy

In [132]:
scipy.spatial.distance.pdist([one_topics, two_topics])

array([0.30274955])

In [133]:
scipy.spatial.distance.pdist([one_topics, three_topics])

array([0.4921023])

In [134]:
scipy.spatial.distance.pdist([two_topics, three_topics])

array([0.37748321])

In [135]:
distances = 'braycurtis', 'canberra', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule'

In [137]:
for d in distances:
    a = scipy.spatial.distance.pdist([one_topics, two_topics], d)
    x = scipy.spatial.distance.pdist([two_topics, three_topics], d)
    y = scipy.spatial.distance.pdist([one_topics, three_topics], d)
    print(d, a / ((x + y)/2))

braycurtis [0.70863555]
canberra [0.75452321]
chebyshev [0.55158917]
cityblock [0.70863561]
correlation [0.28397451]
cosine [0.29919385]
dice [nan]
euclidean [0.6963077]
hamming [1.]
jaccard [1.]
jensenshannon [0.74923541]
kulsinski [nan]


/home/probinso/git/Tiffany/lda/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


ValueError: The number of observations (2) is too small; the covariance matrix is singular. For observations with 100 dimensions, at least 101 observations are required.

In [138]:
import numpy as np

In [147]:
x = np.asarray([7,8, 9, 0])
idx = np.asarray([], dtype=np.int)
jdx = np.asarray([2,3])

In [148]:
x[idx]

array([], dtype=int64)